# Bailey Pairs and Chain Discovery

A **Bailey pair** relative to $a$ is a pair of sequences $(\alpha_n, \beta_n)$ satisfying:

$$\beta_n = \sum_{r=0}^{n} \frac{\alpha_r}{(q;q)_{n-r}(aq;q)_{n+r}}$$

The **Bailey lemma** (also called **Bailey's transform**) shows that from one Bailey
pair, one can derive infinitely many others by iterating a transformation with
parameters $b$ and $c$. This iteration produces a **Bailey chain**.

Bailey pairs and chains are the engine behind many partition identities, including
the celebrated Rogers-Ramanujan identities.

## The Weak Bailey Lemma

The **weak Bailey lemma** states that if $(\alpha_n, \beta_n)$ is a Bailey pair
relative to $a$, then:

$$\sum_{n \geq 0} q^{n^2} a^n \beta_n = \frac{1}{(aq;q)_\infty} \sum_{n \geq 0} q^{n^2} a^n \alpha_n$$

Let's verify this with the **Rogers-Ramanujan** Bailey pair and $a = q$:

In [1]:
from q_kangaroo import QSession, bailey_weak_lemma

s = QSession()

# Verify the weak Bailey lemma with the Rogers-Ramanujan pair, a = q
lhs, rhs = bailey_weak_lemma(s, "rogers-ramanujan", 1, 1, 1, 10, 20)
print("LHS:", lhs)
print("RHS:", rhs)
print("LHS == RHS:", repr(lhs) == repr(rhs))

LHS: 1 + q^2 + q^3 + q^4 + q^5 + 2*q^6 + 2*q^7 + 3*q^8 + 3*q^9 + 4*q^10 + 4*q^11 + 6*q^12 + 6*q^13 + 8*q^14 + 9*q^15 + 11*q^16 + 12*q^17 + 15*q^18 + 16*q^19 + O(q^20)
RHS: 1 + q^2 + q^3 + q^4 + q^5 + 2*q^6 + 2*q^7 + 3*q^8 + 3*q^9 + 4*q^10 + 4*q^11 + 6*q^12 + 6*q^13 + 8*q^14 + 9*q^15 + 11*q^16 + 12*q^17 + 15*q^18 + 16*q^19 + O(q^20)
LHS == RHS: True


Both sides match, confirming the weak Bailey lemma for this pair.

## Applying the Bailey Lemma

The full **Bailey lemma** transforms a Bailey pair $(\alpha_n, \beta_n)$ relative
to $a$ into a new pair $(\alpha_n', \beta_n')$ using additional parameters $b$ and $c$.
This is the core step that enables Bailey chains.

The `bailey_apply_lemma` function applies one step of this transformation:

In [2]:
from q_kangaroo import bailey_apply_lemma

# Apply Bailey lemma to the unit pair with a=q^2, b=q/2, c=q/3
derived = bailey_apply_lemma(s, "unit", (1,1,2), (1,2,1), (1,3,1), 4, 15)
derived

{'name': "lemma(unit, b=QMonomial { coeff: QRat(1/2), power: 1 }, c=QMonomial { coeff: QRat(1/3), power: 1 })", 'pair_type': 'tabulated', 'num_terms': 5}

The result describes the derived Bailey pair: a tabulated pair with 5 terms,
obtained by applying the lemma to the unit pair.

## Building a Bailey Chain

A **Bailey chain** applies the lemma iteratively, producing a sequence of
derived pairs. Each step produces new partition identities.

The `bailey_chain` function builds a chain of specified depth:

In [3]:
from q_kangaroo import bailey_chain

# Build a depth-2 Bailey chain from the unit pair
chain = bailey_chain(s, "unit", (1,1,2), (1,2,1), (1,3,1), 2, 4, 15)

print(f"Chain length: {len(chain)}")
for step in chain:
    print(f"Step {step['index']}: {step['name']} ({step['pair_type']})")

Chain length: 3
Step 0: unit (Unit)
Step 1: lemma(unit, b=QMonomial { coeff: QRat(1/2), power: 1 }, c=QMonomial { coeff: QRat(1/3), power: 1 }) (Tabulated)
Step 2: lemma(lemma(unit, b=QMonomial { coeff: QRat(1/2), power: 1 }, c=QMonomial { coeff: QRat(1/3), power: 1 }), b=QMonomial { coeff: QRat(1/2), power: 1 }, c=QMonomial { coeff: QRat(1/3), power: 1 }) (Tabulated)


The chain starts from the unit pair (step 0) and applies the lemma twice,
producing two derived tabulated pairs.

## Automated Bailey Pair Discovery

Given two series, `bailey_discover` searches the database to determine whether
they are related by a Bailey pair:

In [4]:
from q_kangaroo import bailey_discover

# Verify that the two sides of the weak Bailey lemma are recognized
result = bailey_discover(s, lhs, rhs, (1,1,1), 2, 20)

print(f"Found: {result['found']}")
print(f"Verification: {result['verification']}")

Found: True
Verification: direct equality


The discovery engine confirms the two series are equal (direct equality),
which is expected since they are both sides of a verified Bailey lemma identity.

## Significance for Partition Identities

Bailey pairs and chains provide a systematic machine for generating partition
identities:

- The **Rogers-Ramanujan identities** (1894) follow from the Rogers-Ramanujan
  Bailey pair applied through the weak lemma.
- **Andrews' multiple series identities** arise from iterating Bailey chains.
- **Slater's list** of 130 Rogers-Ramanujan-type identities was generated by
  systematically exploring Bailey pairs.

The q-Kangaroo database includes three canonical Bailey pairs:
- `"unit"` -- the trivial pair ($\alpha_0 = 1$, $\alpha_n = 0$ for $n > 0$)
- `"rogers-ramanujan"` -- produces the Rogers-Ramanujan identities
- `"q-binomial"` -- related to $q$-binomial coefficient identities

Each can be fed into the Bailey lemma and chain machinery to generate
families of $q$-series identities.